In [1]:
data_var = '2023-12-21'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8434,2023-12-21,Brasil Nbb,19:00,Cerrado,São Paulo,3.02,1.35,155.5,1.86,1.84,5.5,2.00,1.72,AuXmpG4F,0.331126,0.740741,0.537634,0.543478,0.071867,0.2,0.4,NaN,NaN,410.540,255.312996,0.621896,117.316,39.678602,0.338220,155.278,117.402,179.20,105.30,0.0,0.0,0.0,0.0,0.540443,0.007644,0.106446,-1.91,-0.382,-5.287958,0.0,0.0,0.0,-0.75,-0.150,-2.333333,0.0,0.0,0.0
8435,2023-12-21,Brasil Nbb,19:30,União Corinthians,Corinthians Paulista,2.53,1.48,156.5,1.87,1.79,3.5,2.00,1.72,WlYqodk9,0.395257,0.675676,0.534759,0.558659,0.070933,0.0,0.6,NaN,NaN,304.778,125.778162,0.412688,147.470,90.083122,0.610857,151.936,204.078,244.94,120.00,0.0,0.0,0.0,0.0,0.370305,0.030912,0.106446,-3.63,-0.726,-2.107438,0.0,0.0,0.0,-3.50,-0.700,-0.685714,0.0,0.0,0.0
8436,2023-12-21,Brasil Nbb,20:00,Caxias do Sul,Pinheiros,2.12,1.67,149.5,1.83,1.87,1.5,2.00,1.72,8URNwEsq,0.471698,0.598802,0.546448,0.534759,0.070501,1.0,0.8,NaN,NaN,240.726,56.236132,0.233611,142.338,36.430499,0.255944,183.308,252.252,195.92,240.12,0.0,0.0,0.0,0.0,0.167915,0.015289,0.106446,-3.59,-0.718,-1.559889,0.0,0.0,0.0,-0.13,-0.026,-25.769231,0.0,0.0,0.0
8437,2023-12-21,Brasil Nbb,20:00,Mogi,Franca,6.15,1.10,161.5,1.85,1.85,11.5,2.10,1.65,jkudrfZR,0.162602,0.909091,0.540541,0.540541,0.071693,0.8,0.4,NaN,NaN,242.664,73.357240,0.302300,103.364,18.395632,0.177969,274.898,118.722,230.55,101.64,0.0,0.0,0.0,0.0,0.985073,0.000000,0.169706,-3.07,-0.614,-8.387622,0.0,0.0,0.0,-0.45,-0.090,-1.111111,0.0,0.0,0.0
8438,2023-12-21,Brasil Nbb,21:00,Flamengo,Fortaleza B.C.,1.16,4.76,155.5,1.86,1.84,-12.5,2.06,1.68,h4MhqzKL,0.862069,0.210084,0.537634,0.543478,0.072153,0.6,0.8,NaN,NaN,122.574,23.012384,0.187743,154.324,106.962872,0.693106,109.958,246.188,162.75,145.34,0.0,0.0,0.0,0.0,0.859995,0.007644,0.143690,1.03,0.206,0.776699,0.0,0.0,0.0,0.97,0.194,19.381443,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8523,2023-12-21,Rússia Liga Vtb United,13:00,Unics Kazan,Minsk,1.03,40.00,156.5,1.80,1.96,-29.5,2.03,1.75,hYmtIsbs,0.970874,0.025000,0.555556,0.510204,-0.004126,0.6,0.0,NaN,NaN,113.970,17.262374,0.151464,465.926,251.886855,0.540616,103.090,0.000,101.01,584.80,0.0,1.0,0.0,0.0,1.343210,0.060179,0.104757,-1.58,-0.316,-0.094937,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
8524,2023-12-21,Rússia Liga Vtb United,13:30,Zenit Petersburg,Astana,1.05,10.50,156.5,1.86,1.92,-18.5,2.03,1.73,jP1EMNkQ,0.952381,0.095238,0.537634,0.520833,0.047619,0.0,0.0,NaN,NaN,128.982,49.326949,0.382433,263.248,158.715583,0.602913,0.000,0.000,112.32,125.49,0.0,0.0,0.0,0.0,1.157084,0.022448,0.112836,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
8525,2023-12-21,Rússia Liga Vtb United,14:00,Lokomotiv Kuban,Uralmash Ekaterinburg,1.24,4.15,162.5,1.81,1.97,-10.5,2.00,1.77,xUJL5DhD,0.806452,0.240964,0.552486,0.507614,0.047415,0.0,0.8,NaN,NaN,118.460,25.376623,0.214221,199.392,37.737193,0.189261,0.000,158.622,119.70,181.30,0.0,0.0,0.0,0.0,0.763518,0.059861,0.086278,0.00,0.000,inf,0.0,0.0,0.0,0.25,0.050,63.000000,0.0,0.0,0.0
8526,2023-12-21,Uruguai Liga Uruguaia,21:00,Penarol,Urupan,1.05,8.55,161.5,1.88,1.79,-17.5,1.94,1.77,tfMNfRfS,0.952381,0.116959,0.531915,0.558659,0.069340,0.4,0.0,NaN,NaN,153.546,44.687851,0.291039,213.216,36.478603,0.171088,164.050,0.000,94.34,270.28,0.0,0.0,0.0,0.0,1.104854,0.03

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
6,21:00,Eua Nba,Detroit Pistons,Utah Jazz,236.5,1.89,1.0000,Over
7,22:00,Eua Nba,Chicago Bulls,San Antonio Spurs,229.5,1.86,1.0000,Over
9,22:00,Eua Nba,Milwaukee Bucks,Orlando Magic,238.5,1.97,0.9993,Over
47,16:00,Eua Ncaa,Lafayette,Quinnipiac,138.5,1.91,0.8583,Over
49,16:00,Eua Ncaa,Texas Tech,Texas Arlington,144.5,1.91,0.8782,Over
60,21:00,Eua Ncaa,Arkansas,Abilene Christian,149.5,1.91,0.8768,Over
61,22:00,Eua Ncaa,Houston,Texas State,125.5,1.91,0.9518,Over
62,22:00,Eua Ncaa,Iowa State,Eastern Illinois,138.5,1.91,0.9562,Over
63,22:00,Eua Ncaa,Miami,Stonehill,152.5,1.91,0.9491,Over
